In [1]:
#encode the 2VYR sequence
sequence = "EVQLLESGGGLVQPGGSLRLSCAASGFTFEEYAMLWVRQAPGKGLEWVSGINARGYTTYYADSVKGRFTISRDNSKNTLYLQMNSLRTEDTAVYYCAKPWYPFMASKGSEFDYWGQGTLVTVSSAAALEIKRASQPELAPEDPEDVEHHHHHH"

# Mapping each amino acid to a MIDI note
aa_to_note = {
    'A': 60, 'C': 61, 'D': 62, 'E': 63, 'F': 64,
    'G': 65, 'H': 66, 'I': 67, 'K': 68, 'L': 69,
    'M': 70, 'N': 71, 'P': 72, 'Q': 73, 'R': 74,
    'S': 75, 'T': 76, 'V': 77, 'W': 78, 'Y': 79,
}

# This one finds the AA spots that trigger the second instrument
def find_instrument2_segments(seq):
    segments = []
    i = 0
    while i < len(seq) - 1:
        if seq[i:i+2] == 'AA':
            start = i
            end = start + 7
            found_y = False
            for j in range(start + 7, min(start + 18, len(seq))):
                if seq[j] == 'Y':
                    end = j + 1
                    found_y = True
                    break
            if not found_y:
                end = min(start + 18, len(seq))
            segments.append((start, end))
            i = start + 1
        else:
            i += 1
    return segments

# This one finds all HHH stretches and continues until it hits something that's not H
def find_hhh_segments(seq):
    segments = []
    i = 0
    while i < len(seq) - 2:
        if seq[i:i+3] == 'HHH':
            start = i
            end = i
            while end < len(seq) and seq[end] == 'H':
                end += 1
            segments.append((start, end))
            i = end
        else:
            i += 1
    return segments


In [3]:
!pip install MIDIUtil

from midiutil import MIDIFile

# Create MIDI with 3 tracks this time: piano, violin, and drums
mf = MIDIFile(3)
mf.addTempo(0, 0, 120)
mf.addTempo(1, 0, 120)
mf.addTempo(2, 0, 120)

# Piano (full sequence)
mf.addProgramChange(0, 0, 0, 0)   # Piano
# Violin (AA-triggered segments)
mf.addProgramChange(1, 1, 0, 40)  # Violin
# Drums are on channel 9 — we don’t need to set an instrument, it’s built-in

# --- Instrument 1: Piano playing full sequence ---
time = 0
for aa in sequence:
    if aa in aa_to_note:
        note = aa_to_note[aa]
        mf.addNote(track=0, channel=0, pitch=note, time=time, duration=1, volume=80)
    time += 1

# --- Instrument 2: Violin playing from AA to Y or 18 aa max ---
segments = find_instrument2_segments(sequence)

for start, end in segments:
    time = start
    for i in range(start, end):
        aa = sequence[i]
        if aa in aa_to_note:
            note = aa_to_note[aa]
            mf.addNote(track=1, channel=1, pitch=note, time=time, duration=1, volume=100)
        time += 1

# --- Instrument 3: Drums under HHH segments ---
hhh_segments = find_hhh_segments(sequence)

for start, end in hhh_segments:
    time = start
    for i in range(start, end):
        # Snare drum = 38
        mf.addNote(track=2, channel=9, pitch=38, time=time, duration=1, volume=120)
        time += 1

# Save it
with open("2VYR.mid", "wb") as output_file:
    mf.writeFile(output_file)

print("✅ MIDI file '2VYR.mid' created!")

✅ MIDI file '2VYR.mid' created!
